In [10]:
import zmq
import json
import math
import numpy as np
import transforms3d as tranf

context = zmq.Context()
socket = context.socket(zmq.REQ)
socket.connect("tcp://localhost:3001")

<SocketContext(connect='tcp://localhost:3001')>

In [12]:
LED_REAL_DISTANCE=0.5
BASELINE=2.3

def is_unknown_cmd(m):
    return b'Unknown command' in m

def get_led(socket):
    socket.send_string('get_led')
    message = socket.recv_json()
    return np.array(message['led'])


In [44]:
socket.send_string("get_config")
config = socket.recv_json()
print(config)
fov_angle = config['camera']['fov']*math.pi/180
size_pixel = config['camera']['size_pixel']
leds = get_led(socket)
print(leds)

{'camera': {'base_folder': '../../data', 'file': {'off': 'mid_off.tiff', 'on': 'mid_on.tiff'}, 'fov': 30, 'size_pixel': [400, 300]}}
[[142.51869159 341.97368421]
 [136.07386364 462.82054795]]


In [45]:
from transforms3d.quaternions import quat2mat, qinverse, qmult
def qdiff(src, dst):
    return qmult(src, qinverse(dst))

quat_deputy = np.array([0.0, 0.0, 0.0, 1.0])

In [48]:

# get relative orientation difference of deputy w.t.t chief
leds = np.array([[1, 0],[1,1]])
socket.send_string("get_orientation")
quat_chief = np.array(socket.recv_json())
quat_diff = qdiff(quat_chief, quat_deputy)
midpoint = np.sum(leds, axis=1)/2
# projective transform
A1 = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 0], [0, 0, 1]], np.float32)
R = np.block([[quat2mat(quat_diff), np.zeros((3, 1))], [np.zeros((1, 3)), 1]])
T = np.block([[np.eye(3), np.r_[-midpoint, 0][:, None]], [np.zeros((1,3)), 1]])
A2 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1]], np.float32)
transform = A2 @ (R @ (T @ A1))
leds_corrected = (transform@np.hstack((leds, np.ones((2, 1)))).T).T[:, :2]
print(leds_corrected)


[[-0.33598463 -0.06440869]
 [ 0.26427758 -0.3001311 ]]


In [9]:
led_sep_pixel = np.linalg.norm(leds[1] - leds[0])
ratio = led_sep_pixel/LED_REAL_DISTANCE
distance = LED_REAL_DISTANCE/2/np.tan(led_sep_pixel/size_pixel[1]*fov_angle/2)
angle = np.arctan2(*(leds[1]-leds[0])[::-1])
offset = (leds[1]+leds[0])/2/ratio
print(distance, angle, offset)


2.3364634905908743 1.6199370612109167 [0.57307786 1.64555213]


In [10]:
R = tranf.axangles.axangle2mat([1,0,0], angle)
T = np.array([BASELINE-distance, *(-offset)])
print(R, T)

[[ 1.          0.          0.        ]
 [ 0.         -0.04912096 -0.99879284]
 [ 0.          0.99879284 -0.04912096]] [-0.03646349 -0.57307786 -1.64555213]
